# 毕业论文的代码
## 题目：市值管理对股票资产定价的影响——基于拓展的Fama-French三因子模型实证分析
作者：包岩霖 南开大学商学院财务管理专业本科生

邮箱：byljason12308@gmail.com

In [ ]:
# 导入所需包
import time
import numpy as np
import pandas as pd
import statsmodels.api as sm
import tushare as ts
import matplotlib as mpl
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from datetime import datetime

mpl.rc('font',family='Times New Roman')
# 此处为注册tushare获取的token
ts.set_token('')
pro = ts.pro_api()
%matplotlib inline

## 导入第一部分处理后存储的.npy文件
第一次使用请运行第一部分的代码，之后再使用时请在此步导入数据，略去第一部分，which will save a lot of time!

Tushare是免费开源的python财经数据接口，本文使用的Tushare均可以注册Tushare账号后免费获取使用。

此处因为数据版权问题，不提供数据源文件。

In [ ]:
# 在存储.npy文件和重新读取时对文件名和变量名进行了更新统一，最开始写代码时的原变量名没改主要是不待老改了
dict_pf = np.load('dict_pf.npy', allow_pickle=True).item()
dict_mnt = np.load('dict_mnt.npy', allow_pickle=True).item()
dict_seo = np.load('dict_seo.npy', allow_pickle=True).item()
dict_day = np.load('dict_day.npy', allow_pickle=True).item()
dict_rep = np.load('dict_rep.npy', allow_pickle=True).item()
dict_sbpn_trading = np.load('dict_sbpn_trading.npy', allow_pickle=True).item()
df_fs = pd.read_pickle('df_fs.pkl')
df_rmd = pd.read_pickle('df_rmd.pkl')
df_tmv = pd.read_pickle('df_tmv.pkl')

# 剔除后的股票信息的字典
dict_rmd = {}
for stk in df_rmd.iloc[:,0]:
    dict_rmd[stk] = dict_day[stk]

### CSMAR数据
#### 增发数据

In [ ]:
# 导入2011～2020年10年的A股上市公司增发数据
df_seo = pd.read_excel('/Users/bbbbyl/Python/datasets/RS_Aibasic.xlsx').drop([0,1]).reset_index(drop=True)
# 构建一个dictionary来分别存储每只股票的数据
dict_seo = dict(list(df_seo.groupby('Stkcd')))
# 将数据中的日期部分改为与Tushare数据统一的格式
for stk in dict_seo.keys():
    dict_seo[stk].Ailtdt = [datetime.strftime(datetime.strptime(date, '%Y-%m-%d'),'%Y%m%d') for date in dict_seo[stk].Ailtdt]
# 将字典存至.npy文件，以后直接导入使用
np.save('dict_seo.npy', dict_seo)

### Tushare数据
#### daily数据

In [ ]:
# 首先导入2011年1月4日沪深300指数成分股标签
#stks = pro.index_weight(index_code='399300.SZ', start_date='20110104', end_date='20110104')['con_code']
# 首先获取2011年前上市公司标签
stks = pro.query('stock_basic',exchange='',list_status='L',fields='ts_code,symbol,name,area,industry,list_date')
lst_stks = list(stks[stks['list_date']<'20110101']['ts_code'])
# 获取十年中的交易日
trade_cal = pro.query('trade_cal',start_date='20110101',end_date='20201231')
trade_cal = pd.DataFrame(trade_cal[trade_cal['is_open']==1]['cal_date']).reset_index(drop=True)
# 构建一个dictionary来分别存储每只股票的数据
dict_stks = {}
for stk in lst_stks:
    dict_stks[stk] = ts.pro_bar(ts_code=stk,adj='hfq',start_date='20110101',end_date='20201231')
for stk in dict_stks.keys():
    dict_stks[stk] = pd.merge(trade_cal,dict_stks[stk],left_on='cal_date',right_on='trade_date',how='left')
    dict_stks[stk]['Rtr'] = dict_stks[stk]['pct_chg']/100
    dict_stks[stk]['LogRtr'] = np.log(dict_stks[stk]['Rtr']+1)
# 将字典存至.npy文件，以后直接导入使用
np.save('dict_day.npy', dict_stks)

#### 回购数据

In [ ]:
# 在tushare下载2011～2020年10年间的A股回购数据
df_rep = pd.DataFrame()
for year in range(2011, 2018):
    start = str(year) + '0101'
    end = str(year) + '1231'
    df_rep = df_rep.append(pro.repurchase(ann_date='',start_date=start,end_date=end),ignore_index=True)
    time.sleep(3.01) #数据接口有20次/分的频率限制
for year in range(2018, 2021):
    start = str(year) + '0101'
    end = str(year) + '0630'
    df_rep = df_rep.append(pro.repurchase(ann_date='',start_date=start,end_date=end),ignore_index=True)
    time.sleep(3.01) #数据接口有20次/分的频率限制
    start = str(year) + '0701'
    end = str(year) + '1231'
    df_rep = df_rep.append(pro.repurchase(ann_date='',start_date=start,end_date=end),ignore_index=True)
    time.sleep(3.01) #数据接口有20次/分的频率限制
df_rep = df_rep.sort_values('ann_date').reset_index(drop=True)
# 构建一个dictionary来分别存储每只股票的数据
dict_rep = dict(list(df_rep.groupby('ts_code')))
# 将字典存至.npy文件，以后直接导入使用
np.save('dict_rep.npy', dict_rep)

### 锐思数据
#### Fama-French 三因子因子收益率数据


注：无风险利率使用上海银行间3个月同业拆放利率；使用所有交易所的三因子数据；市场溢酬因子（Rm-Rf）、市值因子（SMB）、账面市值比因子（HML）采用流通市值加权

In [ ]:
# 所用数据的变量名
columns = ['Date', 'Rmrf', 'Smb', 'Hml']
# 导入2011～2020年10年的A股Fama-French三因子因子收益率数据
df_3fs = pd.read_excel('/Users/bbbbyl/Python/datasets/RESSET_THRFACDAT_DAILY_1.xlsx')
df_3fs = df_3fs[(df_3fs['交易所标识_Exchflg']==0) & (df_3fs['市场标识_Mktflg']=='A')].iloc[:,2:6]
df_3fs.columns = columns

#### 无风险利率数据

In [ ]:
# 所用数据的变量名
columns = ['Date', 'Rf']
# 导入2011～2020年10年的无风险利率数据
df_rf = pd.read_excel('/Users/bbbbyl/Python/datasets/RESSET_BDDRFRET_1.xlsx')
df_rf.columns = columns
# 与三因子收益率数据合并
df_fs = pd.merge(df_3fs, df_rf, left_on='Date', right_on='Date', how='left')
# 将数据中的日期部分改为与Tushare数据统一的格式
df_fs.Date = [datetime.strftime(datetime.strptime(date, '%Y-%m-%d'),'%Y%m%d') for date in df_fs.Date]
# 存储为.pkl文件，以后直接导入使用
df_fs.to_pickle('df_fs.pkl')

#### 市值数据

In [ ]:
# 导入2011～2020年10年的年流通市值数据
df_tmv = pd.read_excel('/Users/bbbbyl/Python/datasets/RESSET_YRMV_1.xlsx', dtype={'Stkcd':str})
# 将数据中的日期部分改为与Tushare数据统一的格式
df_tmv['Date'] = [string.replace('-','') for string in df_tmv['Date']]
# 存储为.pkl文件，以后直接导入使用
df_tmv.to_pickle('df_tmv.pkl')

## 第二部分：数据初步探索的可视化
Have fun and be creative!
### 停牌日数量与日收益率之间有关系吗？

In [ ]:
# 准备画图用的ndarrays（此处保留了股票代码，方便日后使用，但是如果单纯为了画出图像，直接生成ndarrays会方便很多，略去了不同数据种类之间的转换）
dict_null = {}
for stk in dict_day.keys():
    dict_null[stk] = dict_day[stk]['Rtr'].isnull().sum()
dict_Rtr = {}
for stk in dict_day.keys():
    dict_Rtr[stk] = dict_day[stk]['Rtr'].mean()
df_null = pd.DataFrame(dict_null,index=pd.Index(['num_nontrade'])).T.reset_index().rename(columns={'index':'ts_code'})
df_Rtr = pd.DataFrame(dict_Rtr,index=pd.Index(['mean_Rtr'])).T.reset_index().rename(columns={'index':'ts_code'})
arr_null = np.array(df_null.iloc[:,1])
arr_Rtr = np.array(df_Rtr.iloc[:,1])

# 那就画图吧！
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(15,10))
fig.patch.set_facecolor('w')
gspec = gridspec.GridSpec(4,4)
gspec.update(wspace=0.15, hspace=0.05)

top_hist = plt.subplot(gspec[0,1:])
side_hist = plt.subplot(gspec[1:,0])
scatter = plt.subplot(gspec[1:,1:])

_ = scatter.scatter(arr_Rtr, arr_null, color='grey', alpha=0.5)
scatter.set_yticks([])
_ = top_hist.hist(arr_Rtr, bins=100, density=True, color='grey')
top_hist.set_xticks([])
_ = side_hist.hist(arr_null, bins=100, density=True, orientation='horizontal', color='grey')
side_hist.invert_xaxis()
_ = scatter.set_xlabel(xlabel='Average Daily Return', fontsize=18)
_ = side_hist.set_ylabel('Number of Suspension Days', fontsize=18)
_ = fig.suptitle('Suspension Days vs Average Daily Return', fontsize=25, fontweight='bold')
# 保存我们的成果吧！
plt.savefig('SusDays.png', dpi=500)

### 剔除累计停牌日大于两年的股票前后个股日收益率的分布

In [ ]:
# 剔除前
fig = plt.figure(figsize=(12,10))
fig.patch.set_facecolor('w')
ax = plt.subplot(1,1,1)
# statsmodels包的Q-Q图
sm.qqplot(arr_Rtr, line='s', ax=ax)
_ = ax.set_title('Q-Q Plot of Average Daily Return', fontsize=20, fontweight='bold')
# 保存一下
plt.savefig('qqplot.png', dpi=500)

In [ ]:
# 剔除过程
# 保留的股票的股票代码
lst_remained = []
for stk in dict_null.keys():
    if dict_null[stk] < 250*2:
        lst_remained.append(stk)
len(lst_remained) # 290
# 存储至本地文件方便之后使用
df_rmd = pd.DataFrame(lst_remained)
df_rmd['rs'] = [string[:-3] for string in df_rmd.iloc[:,0]]
df_rmd.columns = ['Tushare','rs']
df_rmd.to_pickle('df_rmd.pkl')

In [ ]:
# 剔除后
dict_Rtr_remained = {}
for stk in lst_remained:
    dict_Rtr_remained[stk] = dict_Rtr[stk]
df_Rtr_remained = pd.DataFrame(dict_Rtr_remained,index=pd.Index(['mean_Rtr'])).T.reset_index().rename(columns={'index':'ts_code'})
arr_Rtr_remained = np.array(df_Rtr_remained.iloc[:,1])
fig = plt.figure(figsize=(12,10))
fig.patch.set_facecolor('w')
ax = plt.subplot(1,1,1)
sm.qqplot(arr_Rtr_remained, line='s', ax=ax)
_ = ax.set_title('Q-Q Plot of Average Daily Return (Remained)', fontsize=20, fontweight='bold')
# 保存一下
plt.savefig('qqplot_remained.png', dpi=500)

## 第三部分：因子投资组合构建及因子收益率的计算

In [ ]:
# 提取留下的290只股票的增发和回购数据
dict_seo_rmd = {}
dict_rep_rmd = {}
for stk in df_rmd['Tushare']:
    try:
        dict_seo_rmd[stk[:-3]] = dict_seo[stk[:-3]]
    except KeyError:
        pass
    try:
        dict_rep_rmd[stk[:-3]] = dict_rep[stk]
    except KeyError:
        pass

In [ ]:
# 验算格  # 第一次尝试实际运用try except语句，谨慎一点
print(len(dict_seo_rmd),len(dict_rep_rmd))
lst01 = []
lst02 = []
for stk in df_rmd['Tushare']:
    if stk[:-3] in dict_seo.keys():
        lst01.append(stk)
    if stk in dict_rep.keys():
        lst02.append(stk)
print(len(lst01), len(lst02))

In [ ]:
# 创建年频的2X2独立双重排序投资组合
# 建立一个年年份列表
lst_yr = list(range(2011, 2021))
# 计算A股流通市值2011～2020年每年的中位数
lst_tmv_median = []
for year in lst_yr:
    lst_tmv_median.append(df_tmv[df_tmv['Date']==str(year)+'1231']['Yrtmv'].median())
#df_tmv_median = pd.DataFrame(zip(lst_yr, lst_tmv_median), columns=['year', 'tmv_median'])
# 创建一个dictionary储存每只股票的市值分类数据
dict_pf = {}
for stk in df_rmd['rs']:
    lst_temp = []
    for year in lst_yr:
        try:
            lst_temp.append(df_tmv[(df_tmv['Stkcd']==stk)&(df_tmv['Date']==str(year)+'1231')]['Yrtmv'].iloc[0])
        except:
            lst_temp.append(np.nan)
    dict_pf[stk] = pd.DataFrame(zip(lst_yr, lst_temp, lst_tmv_median), columns=['year', 'tmv', 'median'])
    dict_pf[stk]['SB'] = dict_pf[stk]['tmv']<=dict_pf[stk]['median']
    dict_pf[stk]['SB'] = ['S' if b else 'B' for b in dict_pf[stk]['SB']]
    dict_pf[stk].drop(columns=['tmv','median'], inplace=True)
# 将回购和增发数据添加入dict_pf，用于构建PMN因子
for stk in dict_pf.keys():
    lst_seo = ['-'] * 10
    try:
        for date in dict_seo_rmd[stk]['Ailtdt']:
            try:
                lst_seo[int(date[:4])-2010] = 'N'
            except:
                pass
    except KeyError:
        pass
    dict_pf[stk]['seo'] = lst_seo
    lst_rep = ['-'] * 10
    try:
        for date in dict_rep_rmd[stk].query('proc=="实施"')['ann_date']:
            try:
                lst_rep[int(date[:4])-2010] = 'P'
            except:
                pass
    except KeyError:
        pass
    dict_pf[stk]['rep'] = lst_rep
    dict_pf[stk]['PN'] = (dict_pf[stk]['seo']+dict_pf[stk]['rep']).str.replace('-','')
    dict_pf[stk]['SBPN'] = dict_pf[stk]['SB']+dict_pf[stk]['PN']
    dict_pf[stk].drop(columns=['seo','rep','SB','PN'], inplace=True)
    dict_pf[stk]['stk'] = [stk]*10
# 保存至npy文件
np.save('dict_pf.npy', dict_pf)

In [ ]:
# 转换成以年份为key的dictionary
df_pf = pd.DataFrame()
for stk in dict_pf.keys():
    df_pf = pd.concat([df_pf, dict_pf[stk]], ignore_index=True)
dict_pf_yr = dict(list(df_pf.groupby('year')))

In [ ]:
# SBPN分类股票数量统计
SBPN = df_pf.groupby(['year','SBPN']).agg('count')
# 将结果保存至xlsx文件
df_pf.groupby(['year','SBPN']).agg('count').to_excel('SBPN.xlsx')
# 可视化
fig = plt.figure(figsize=(15,8))
ax = plt.subplot(1,1,1)
fig.patch.set_facecolor('w')
_ = SBPN.unstack().plot.barh(stacked=True, ax=ax, alpha=0.85)
_ = ax.set_title('Stacked Bar Chart of SBPN Categories', fontsize=20, fontweight='bold')
# 保存图片
plt.savefig('SBPN_barh.png', dpi=500)

In [ ]:
# 重新收集一下构建PMN投资组合的成分股2019～2020年股票的收益率
# 偷了个懒，求轻喷
lst_sbpn = ['BP','BN','SP','SN']
lst_year = [2019,2020]
def get_sbpn_stks(lst_year: list, lst_sbpn: list) -> dict:
    dict_outer = {}
    for year in lst_year:
        dict_inner = {}
        for sbpn in lst_sbpn:
            dict_inner[sbpn] = list(dict_pf_yr[year].query('SBPN=='+'"'+str(sbpn)+'"')['stk'])
        dict_outer[year] = dict_inner
    return dict_outer
def get_sbpn_trading_data(lst_year:list, lst_sbpn:list) -> dict:
    dict_sbpn_stks = get_sbpn_stks(lst_year,lst_sbpn)
    dict_outer = {}
    for year in lst_year:
        dict_middle = {}
        trade_cal = pro.query('trade_cal',start_date=str(year)+'0101',end_date=str(year)+'1231')
        trade_cal = pd.DataFrame(trade_cal[trade_cal['is_open']==1]['cal_date']).reset_index(drop=True)
        for sbpn in lst_sbpn:
            dict_inner = {}
            for stk in dict_sbpn_stks[year][sbpn]:
                if stk[0] == '6':
                    stk = stk + '.SH'
                else:
                    stk = stk + '.SZ'
                dict_inner[stk[:-3]] = ts.pro_bar(ts_code=stk,adj='hfq',start_date=str(year)+'0101',end_date=str(year)+'1231')
                stk = stk[:-3]
                dict_inner[stk] = pd.merge(trade_cal,dict_inner[stk],left_on='cal_date',right_on='trade_date',how='left')
                dict_inner[stk]['Rtr'] = dict_inner[stk]['pct_chg']/100
                dict_inner[stk]['LogRtr'] = np.log(dict_inner[stk]['Rtr']+1)
            dict_middle[sbpn] = dict_inner
        dict_outer[year] = dict_middle
    return dict_outer
dict_sbpn_trading = get_sbpn_trading_data(lst_year, lst_sbpn)
# 保存一下，从tushare取一次太慢了，下次一定不了。。。
np.save('dict_sbpn_trading.npy', dict_sbpn_trading)

In [ ]:
# SBPN各类投资组合收益率描述性统计分析
def get_df_sbpn_year(year):
    d = {}
    df_sbpn_year = pd.DataFrame()
    for sbpn in lst_sbpn:
        df = pd.DataFrame()
        df['year'] = [year] * len(pro.query('trade_cal',start_date=str(year)+'0101',end_date=str(year)+'1231').query('is_open==1'))
        df['sbpn'] = [sbpn] * len(df)
        for stk in dict_sbpn_trading[year][sbpn].keys():
            df[stk] = dict_sbpn_trading[year][sbpn][stk]['LogRtr']
        df['pf_logr'] = [np.nanmean(df.iloc[i,2:]) for i in range(0,len(df))]
        df.drop(columns=list(dict_sbpn_trading[year][sbpn].keys()), inplace=True)
        d[sbpn] = df
    for sbpn in lst_sbpn:
        df_sbpn_year = pd.concat([df_sbpn_year, d[sbpn]])
    return df_sbpn_year

df_2019 = get_df_sbpn_year(2019)
df_2020 = get_df_sbpn_year(2020)
df_sbpn_pf = pd.concat([df_2019, df_2020])
# 保存到xlsx文件
df_sbpn_pf.groupby(['year','sbpn']).agg(['sum','mean', 'std']).to_excel('sbpn_pf_describe.xlsx')

In [ ]:
# 计算PMN因子投资组合收益率
# 写的时候有点心急了，所以用了俩临时变量，以后注意。。。
d = {}
for year in lst_year:
    df = pd.DataFrame()
    trade_cal = pro.query('trade_cal',start_date=str(year)+'0101',end_date=str(year)+'1231')
    trade_cal = trade_cal[trade_cal['is_open']==1]['cal_date'].reset_index(drop=True)
    df['Date'] = trade_cal
    for sbpn in lst_sbpn:
        df[sbpn] = df_sbpn_pf.query('year=='+str(year)+'&sbpn=='+'"'+sbpn+'"')['pf_logr']
    df['PMN'] = 0.5*(df['BP']+df['SP'])-0.5*(df['BN']+df['SN'])
    df.drop(columns=lst_sbpn, inplace=True)
    d[year] = df
df_PMN = pd.concat([d[2019],d[2020]]).reset_index(drop=True)
# 存进因子dataframe
df_4fs = df_fs.query('Date>="20190101"').reset_index(drop=True)
df_4fs['Pmn'] = df_PMN['PMN']

In [ ]:
# 做图看看咱这PMN因子投资组合收益率这两年长啥样
fig = plt.figure(figsize=(15,6))
ax = plt.subplot(1,1,1)
fig.patch.set_facecolor('w')
_ = sns.lineplot(x=df_PMN['Date'],y=df_PMN['PMN'],ax=ax)
_ = ax.set_xticks([])
_ = ax.set_title('Return of PMN from 2019 to 2020', fontsize=20, fontweight='bold')
# 保存一哈
plt.savefig('PMN.png', dpi=500)

## 第四部分：因变量的构建及回归

In [ ]:
# 设置随机数种子，为了论文的可复现
random.seed(1234)
# 开始随机抽取股票构建投资组合
df_results = pd.DataFrame()
for i in range(1,26):
    stk_index = []
    stks = []
    for j in range(500): # 随机不放回抽取500支股票
        df = pd.DataFrame()
        stk_index.append(random.randint(1,len(df_rmd)-1))
    for k in stk_index:
        stk = df_rmd.iloc[k,0]
        stks.append(stk)
        df[stk] = dict_day[stk].query('cal_date>="20190101"')['LogRtr']
    df['pf_logr'] = [np.nanmean(df.iloc[i,:]) for i in range(0,len(df))]
    df.drop(columns=stks, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = pd.merge(df, df_4fs, left_index=True, right_index=True,how='left')
    df['pf_elogr'] = df['pf_logr'] - df['Rf']
    # 使用statsmodels包进行OLS回归
    results1 = smf.ols('pf_elogr ~ Rmrf + Smb + Hml', data=df).fit()
    results2 = smf.ols('pf_elogr ~ Rmrf + Smb + Hml + Pmn', data=df).fit()
    lst = [results1.rsquared, results2.rsquared, results2.tvalues['Pmn'], results2.pvalues['Pmn']]
    df_results['portfolio'+str(i)] = lst
results_index = pd.Index(['3fs_rsquared','4fs_rsquared','Pmn_t','Pmn_p'])
df_results.index = results_index
# 将结果保存至xlsx文件
df_results.T.to_excel('results.xlsx')

感谢阅读，欢迎批评指正！